<a href="https://colab.research.google.com/github/Servetvrll/customer-segmentation-rfm/blob/kmeans-segmentation/custemer_segmantation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Customer_Segmentation/custemer_segmentation.csv")

print(data.head())
print("-----------------------------------------")
print(data.tail())
print("-----------------------------------------")
print(data.info())
print("-----------------------------------------")
print(data.describe())
print("-----------------------------------------")
print(data.isnull().sum())
print("-----------------------------------------")

data.drop(["CUST_ID"], axis=1, inplace=True)
print(data.head())

In [ ]:
data["MINIMUM_PAYMENTS"] = data["MINIMUM_PAYMENTS"].fillna(0)
data.dropna(subset=['CREDIT_LIMIT'],inplace=True)
data.isnull().sum()

In [ ]:
data.hist(figsize=(20, 15), bins=50)
plt.tight_layout()
plt.show()

corr_matrix = data.corr()

plt.figure(figsize=(18, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Değişkenler Arasındaki Korelasyon Matrisi')
plt.show()

plt.figure(figsize=(15,60))
for i,col in enumerate(data.columns,1):
  plt.subplot(12, 2, i)
  sns.boxplot(x=data[col])
  plt.title(f'{col} İçin Kutu Grafiği')
plt.tight_layout()
plt.show()

In [ ]:
colomns =['BALANCE',  'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE','ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS'
       ]

df_new = data[colomns]

print("Aykırı Değerleri Kırpma İşlemi (IQR Yöntemiyle):")
for col in df_new:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = data[col].clip(lower_bound, upper_bound)
    print(f"'{col}' sütunundaki aykırı değerler [{lower_bound:.2f}, {upper_bound:.2f}] aralığına kırpıldı.")
print("---------------------")

plt.figure(figsize=(15,60))
for i,col in enumerate(data.columns,1):
  plt.subplot(12, 2, i)
  sns.boxplot(x=data[col])
  plt.title(f'{col} İçin Kutu Grafiği')
plt.tight_layout()
plt.show()

In [ ]:
numeric_data = data.select_dtypes(include=[np.number])

corr_matrix = numeric_data.corr().abs()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
mask = mask | (corr_matrix < 0.80)

plt.figure(figsize=(18, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", mask=mask)
plt.title('Korelasyonu 0.80 ve Üzeri Olan Sütunlar')
plt.show()

print(data.columns)

In [ ]:
df = data.drop(['MINIMUM_PAYMENTS', 'PURCHASES_TRX', 'CASH_ADVANCE_TRX'],axis=1)
df.columns


In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("Standartlaştırılmış verinin ilk 5 satırı:")

x = data.copy()
x_scaled = scaler.fit_transform(x)
x_scaled_df = pd.DataFrame(x_scaled, columns=x.columns)

print("Standartlaştırılmış verinin ilk 5 satırı:")


In [ ]:
from sklearn.cluster import KMeans

wcss = []
k_values = range(1, 11)

for k in k_values:

    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(x_scaled_df)
    wcss.append(kmeans.inertia_)


plt.figure(figsize=(10, 6))
plt.plot(k_values, wcss, marker='o', linestyle='--')
plt.title('Dirsek Yöntemi (Elbow Method)')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Küme İçi Kareler Toplamı (WCSS)')
plt.xticks(k_values)
plt.grid(True)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

wcss = []
k_values = range(1, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X_scaled_df)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_values, wcss, marker='o', linestyle='--')
plt.title('Dirsek Yöntemi (Elbow Method)')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Küme İçi Kareler Toplamı (WCSS)')
plt.xticks(k_values)
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
k_values = range(2, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    cluster_labels = kmeans.fit_predict(X_scaled_df)
    score = silhouette_score(X_scaled_df, cluster_labels)
    silhouette_scores.append(score)


plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, marker='o', linestyle='--')
plt.title('Silüet Skoru Yöntemi')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Ortalama Silüet Skoru')
plt.xticks(k_values)
plt.grid(True)
plt.show()

optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
print(f"En yüksek Silüet skorunu veren k değeri: {optimal_k}")
print(f"En yüksek Silüet Skoru: {max(silhouette_scores):.2f}")

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
k_values = range(2, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    cluster_labels = kmeans.fit_predict(x_scaled_df)
    score = silhouette_score(x_scaled_df, cluster_labels)
    silhouette_scores.append(score)


plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, marker='o', linestyle='--')
plt.title('Silüet Skoru Yöntemi')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Ortalama Silüet Skoru')
plt.xticks(k_values)
plt.grid(True)
plt.show()

optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
print(f"En yüksek Silüet skorunu veren k değeri: {optimal_k}")
print(f"En yüksek Silüet Skoru: {max(silhouette_scores):.2f}")